Documnet

Document ek basic data structure hai jo tumhara text (jaise PDF ka content, text file, web page, database record, etc.) ko represent karta hai.

Har Document ke 2 main parts hote hain:

1:page_content → asal text/content jo tum use karna chahte ho

2:metadata → extra information (source, page number, file name, URL, etc.)

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata= {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content = "Cats are independent pets that often enjoy their own space.",
        metedata = {"source":"mammal-pets-doc"}
    ),
    Document(
        page_content = "Goldfish are popular pets for beginners, requiring relatively simple care.",
        metedata = {"source": "mammel-pets-doc"}
    ),
    Document(
        page_content = "Parrots are intelligent birds capable of mimicking human speech.",
        metedata = {"source":"bird-pets-doc"}
    ),
    
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [29]:
from langchain_groq import ChatGroq
llm = ChatGroq (model="llama-3.1-8b-instant",groq_api_key=os.environ["GROQ_API_KEY"])
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E71356BBF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E713568F80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2" )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Prime Laptops\Desktop\Gen\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Prime Laptops\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Vectore Store 

In [14]:
from langchain_chroma import Chroma
db = Chroma.from_documents(documents,embeddings)
db

In [15]:
db.similarity_search("cat")

[Document(id='24e6087f-ab8a-42cf-8750-e996ea1a5228', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='6bb7be06-8998-42b4-8132-aa4bb0016710', metadata={}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='d085dc88-155b-4b73-929a-c960881fcab0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='973bbf7f-023c-4289-9b0e-307ce4c613f3', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

Retrievers

In [18]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retrievel = RunnableLambda(db.similarity_search).bind(k=1)
retrievel.batch(["cat","dog"])

[[Document(id='6bb7be06-8998-42b4-8132-aa4bb0016710', metadata={}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='d085dc88-155b-4b73-929a-c960881fcab0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
r = db.as_retriever(
    search_type = "similarity",
    search_kwargs={"k":1}
)
r.batch(["cat","dog"])

[[Document(id='6bb7be06-8998-42b4-8132-aa4bb0016710', metadata={}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='d085dc88-155b-4b73-929a-c960881fcab0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [30]:
## RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":r,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)


Dogs are great companions, known for their loyalty and friendliness.
